In [50]:
import os

In [51]:
%pwd

'd:\\Projects'

In [54]:
os.chdir("../")

In [55]:
%pwd

'd:\\Projects'

In [56]:
os.chdir("Text-Summarizer")

In [57]:
%pwd

'd:\\Projects\\Text-Summarizer'

In [58]:
from dataclasses import dataclass
from pathlib import Path  ##this is we are creating entity to get return type of a function

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_URL:str                # This we have made a return type
  local_data_file:Path
  unzip_dir:Path

In [59]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [60]:
class ConfigurationManager:  ## preparing method for data ingestion
  def __init__(     ##creating constructor 
      self,
      config_filepath = CONFIG_FILE_PATH,
      param_filepath = PARAMS_FILE_PATH):

      self.config = read_yaml(config_filepath)   ##read this yaml file and access all the variable inside
      self.params = read_yaml(param_filepath)

      ##create artifacts folder using create_dir function  
      create_directories([self.config.artifacts_root]) 



  def get_data_ingestion_config(self) -> DataIngestionConfig:  ## this is the return type      
    # this is data ingestion related config to access all the thing under config.yaml file 
    config = self.config.data_ingestion

    create_directories([config.root_dir])

    data_ingestion_config = DataIngestionConfig(
      root_dir= config.root_dir,
      source_URL= config.source_URL,
      local_data_file= config.local_data_file,
      unzip_dir= config.unzip_dir
    )

    return data_ingestion_config

In [61]:
# Updating components

import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [62]:
class DataIngestion:
  def __init__(self, config: DataIngestionConfig):
    self.config = config



  def download_file(self):
    if not os.path.exists(self.config.local_data_file):
      filename, headers = request.urlretrieve(
        url = self.config.source_URL,
        filename= self.config.local_data_file
      )
      logger.info(f"{filename} download! with the following info: \n{headers}")
    else:
      logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


  def extract_zip_file(self):
    """ 
    zip_file_path : str
    extracts the zip file into the data directory
    Function return None
    """

    unzip_path = self.config.unzip_dir
    os.makedirs(unzip_path, exist_ok=True)
    with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
      zip_ref.extractall(unzip_path)



In [63]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-09 17:30:44,546: INFO: common: yaml:config\config.yamlloaded successfully]
[2025-05-09 17:30:44,551: INFO: common: yaml:params.yamlloaded successfully]
[2025-05-09 17:30:44,552: INFO: common: created directory at: artifacts]
[2025-05-09 17:30:44,556: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-09 17:30:48,342: INFO: 747062685: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 42DD:321AE5:152CD9:344BF4:681DEE70
Accept-Ranges: bytes
Date: Fri, 09 May 2025 12:00:49 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10238-MAA
X-Cache: MISS
X-Cache

In [64]:
import os
print("Current Working Directory:", os.getcwd())
print("Expected path:", os.path.abspath("config/config.yaml"))
print("File exists:", os.path.exists("config/config.yaml"))


Current Working Directory: d:\Projects\Text-Summarizer
Expected path: d:\Projects\Text-Summarizer\config\config.yaml
File exists: True
